# MPI-AMRVAC dataset analysis with yt

This notebook contains examples and instructions specific to the MPI-AMRVAC frontend in yt. 

For a (much) more detailed guide we refer to [the yt cookbook](https://yt-project.org/doc/cookbook/index.html). An overview of the latest news and updates regarding our frontend can be found on [the AMRVAC website](http://amrvac.org/md_doc_yt_usage.html).

Niels Claes and Clément Robert are the current maintainers of both this notebook and the AMRVAC frontend. If you have any ideas or requests for stuff to add here, feel free to contact us. If you encounter issues or bugs regarding the frontend itself, send us a message through the [yt Slack](https://yt-project.org/doc/help/index.html#go-on-slack-or-irc-to-ask-a-question) (we have a dedicated **#amrvac** channel) or file [an issue on GitHub](https://github.com/yt-project/yt/issues).

Please make sure to mention the AMRVAC frontend in the header, and feel free to ping either Niels ([@n-claes](https://github.com/n-claes)) or Clément ([@neutrinoceros](https://github.com/neutrinoceros)) in the issue description!